# [手把手教你完成一个数据科学小项目（6）：城市提取与可视化](https://zhuanlan.zhihu.com/p/42532934)
请先阅读[“中国年轻人正带领国家走向危机”，这锅背是不背？](https://zhuanlan.zhihu.com/p/41880873)  一文，以对“手把手教你完成一个数据科学小项目”系列有个全局性的了解。

截至目前我们已经完成了[数据爬取](https://zhuanlan.zhihu.com/p/42060094)、[数据提取与IP查询](https://zhuanlan.zhihu.com/p/42151036)、[数据异常与清洗](https://zhuanlan.zhihu.com/p/42244913)、[评论数变化情况分析](https://zhuanlan.zhihu.com/p/42333272)、[省份提取与可视化](https://zhuanlan.zhihu.com/p/42406348)，本文继续对地理信息进行处理，并分别提取出省份和城市数据，从而可以用 pyecharts 进行地图可视化。

## 读取数据

In [1]:
import pandas as pd
df = pd.read_csv('Sina_Finance_Comments_All_20180811_Cleaned.csv',encoding='utf-8')
df.head(2)

,Unnamed: 0,Unnamed: 0.1,No,page,nick,time,content,area,ip,ip_loc,...,channel,hot,level,login_type,media_type,mid,stamp,time_ymd,time_mdh,cmntcount
0,0,20,21,2,我阿訇打钱,2018-08-11 17:15:13,[费解][费解][费解],青海海南藏族自治州,36.100.157.102,"青海省海南州 电信 * GeoIP: Hangzhou, Zhejiang, China *...",...,cj,0,5,0,0,5B6EA91F-24649D66-17A3D8EEA-8C5-8D1,1533978913,2018-08-11,08-11 17,3773
1,1,21,22,2,TKK_Questioning,2018-08-11 17:04:20,那你以为不单身就能避免？,安徽合肥,36.7.153.141,"安徽省合肥市 电信 * GeoIP: Hefei, Anhui, China * China...",...,cj,0,4,0,0,5B6EA690-2407998D-1343F7922-8C5-877,1533978260,2018-08-11,08-11 17,3772


本文只对`'area'`列进行处理，`'ip_loc'`列读者可自行探索，比如提取并分析下移动、电信、联通等的占比等。

In [2]:
df[['area','ip_loc']]

,area,ip_loc
0,青海海南藏族自治州,"青海省海南州 电信 * GeoIP: Hangzhou, Zhejiang, China *..."
1,安徽合肥,"安徽省合肥市 电信 * GeoIP: Hefei, Anhui, China * China..."
2,山西太原,"山西省太原市 电信 * GeoIP: Xian, Shaanxi, China * Chin..."
3,广东广州,"广东省广州市 移动 * GeoIP: Guangzhou, Guangdong, China..."
4,广东深圳,"广东省深圳市 电信 * GeoIP: Shenzhen, Guangdong, China ..."
5,湖南,"湖南省 移动 * GeoIP: Changsha, Hunan, China * China..."
6,四川成都,"四川省成都市 联通 * GeoIP: Chengdu, Sichuan, China * U..."
7,浙江,"浙江省 移动 * GeoIP: Hangzhou, Zhejiang, China * Ch..."
8,广西南宁,"广西自治区河池市 电信 * GeoIP: Nanning, Guangxi, China *..."
9,浙江杭州,"浙江省杭州市 移动 * GeoIP: Hangzhou, Zhejiang, China *..."


## 统计 area
看看`area`列有多少种情况。

In [3]:
area_count = df.groupby('area')['area'].count().sort_values(ascending=False)
area_name = list(area_count.index)
area_values = area_count.values
print(len(area_name),len(area_values))
print(area_count)

337 337
area
北京              319
上海              281
广东广州            176
四川成都            136
广东深圳            131
湖北武汉            113
重庆               96
江苏南京             96
浙江杭州             87
陕西西安             73
福建福州             73
浙江               68
江苏苏州             64
安徽合肥             52
天津               44
山东济南             44
江苏徐州             42
江苏无锡             42
辽宁沈阳             40
山东青岛             40
江西南昌             39
河南郑州             39
香港               38
广东佛山             38
广东               37
湖南长沙             36
云南昆明             31
北京海淀             30
山西太原             30
广西南宁             29
               ... 
广东梅州              1
菲律宾西米沙鄢           1
菲律宾               1
澳大利亚澳大利亚首都领地      1
广东云浮              1
瑞士                1
美国弗吉尼亚州           1
辽宁铁岭              1
安哥拉               1
四川达州              1
四川遂宁              1
河北邢台              1
湖南怀化              1
陕西商洛              1
美国宾夕法尼亚州          1
意大利皮埃蒙特           1
宁夏固原              1
意大利托斯卡纳           1
意大利    

全部3000多条评论数据里，地理信息`area_name`共337个唯一值，为了分别对省份和城市进行统计和可视化，需要从`area_name`里找出可以分离出省份和城市的提取方法，以便`apply`应用到`area`列上。

**读者可在此处暂停思考下，自己的思路是什么？该如何实现？**

In [4]:
print(area_name)

['北京', '上海', '广东广州', '四川成都', '广东深圳', '湖北武汉', '重庆', '江苏南京', '浙江杭州', '陕西西安', '福建福州', '浙江', '江苏苏州', '安徽合肥', '天津', '山东济南', '江苏徐州', '江苏无锡', '辽宁沈阳', '山东青岛', '江西南昌', '河南郑州', '香港', '广东佛山', '广东', '湖南长沙', '云南昆明', '北京海淀', '山西太原', '广西南宁', '广东东莞', '甘肃兰州', '澳大利亚', '内蒙古呼和浩特', '河南开封', '福建', '辽宁大连', '河北', '河北石家庄', '江苏南通', '黑龙江哈尔滨', '湖南', '浙江绍兴', '云南', '山东', '日本', '吉林', '吉林长春', '辽宁盘锦', '浙江宁波', '福建厦门', '河南', '贵州贵阳', '浙江金华', '贵州', '山西晋城', '浙江温州', '山东临沂', '四川', '江苏常州', '河南洛阳', '新疆乌鲁木齐', '广东汕头', '江苏扬州', '山东淄博', '四川内江', '江苏泰州', '福建泉州', '广东中山', '山东烟台', '英国英格兰', '浙江嘉兴', '内蒙古包头', '广西', '湖北宜昌', '浙江湖州', '山东济宁', '河北保定', '海南海口', '广东惠州', '河北廊坊', '江苏连云港', '新加坡', '辽宁', '山东潍坊', '安徽淮北', '山西大同', '广西柳州', '湖北襄阳', '浙江台州', '四川绵阳', '河北邯郸', '江西九江', '河南周口', '安徽芜湖', '浙江丽水', '美国', '宁夏银川', '河南南阳', '河北承德', '河北唐山', '江苏盐城', '陕西宝鸡', '青海西宁', '广东揭阳', '河北秦皇岛', '天津塘沽', '河北沧州', '山西长治', '广东韶关', '湖北', '江西上饶', '广西桂林', '江西宜春', '辽宁朝阳', '黑龙江', '湖南常德', '辽宁营口', '湖北黄冈', '辽宁鞍山', '贵州遵义', '山东聊城', '山西', '河南安阳', '安徽六安', '山西运城', '山东德州', '山东东营', '河北衡水', '

地理信息的处理，算是本系列文章的一大亮点，面对这样略显杂乱的数据，新手小白或许会和古柳一样有些头大，（今天你头大了吗？），回想当初做[《爬取张佳玮138w+知乎关注者：数据可视化》](https://zhuanlan.zhihu.com/p/28890605)项目时，筛选了张佳玮的138万关注中自身100+关注的全部4万多知乎用户后，打算分析和可视化时，看到数据的脏乱程度也是一言难尽，为了给大家一个直观的感受，特意翻出数据来展示下，不知道能否得清：
<img src='images/5-张佳玮-area-GIF.gif'>

鬼知道古柳当初是怎么统计的呢！
<img src='images/5-location-1w-zhihu.jpg'>

现在看来，这回的数据真的算好的了，地理信息都是真实的，不会有用户自定义、瞎填的情况出现；格式较为统一，而且数据量也小，再不济，哪怕手动提取省份和城市也不是不可以...（手动是不可能手动的，这辈子都不可能再手动的）

但有个背景需要交代，古柳在完成这个项目时是全部一起做的，也就是说在一个 jupyter notebook 里一步步从爬虫测试与数据爬取、IP 查询与数据提取、评论数变化情况分析和可视化、BUG 遭遇战与异常处理、以及后续文章将涉及的 emoji 提取、评论内容分析、词云展示、情感分析，还有本文的地理信息处理、经纬度查询、用 BDP 实现评论动态热力图等等。“工作量”也不小，虽说现在写成了系列文章的形式，一个 notebook 掰成了八九瓣，读者可能觉察不到，但“如人饮水，冷暖自知”，推进到这一步，确认过地理信息后，“糟糕，是心肌梗塞的感觉”。好了，扯完犊子，还是开始讲下古柳的处理思路吧。

### 数据处理思路
首先再次明确下这次的目的是提取出省份和城市信息，且由于数据量不大，所以后续只在中国地图上进行可视化，因而海外地理信息统一可以筛选出去，实现的方式是按照国家名手动构建一个`unchina`的列表，用来存储本次数据里出现的海外国家，然后遍历所有的337条`area_name`元素，包含这些国家名的就添加到`drop`列表里，然后根据其他国内的地理信息的长度分别打印出来，这样数据就清晰多了！过程中可能有些会被误分，需要核查一遍。

In [5]:
area_len_2 = []
area_len_3 = []
area_len_4 = []
area_len_5 = []
unchina = ['英国','美国','日本','瑞士','法国','瑞典','越南','泰国',
           '意大利','加拿大','菲律宾','新加坡','新西兰','伊拉克','爱尔兰','安哥拉',
           '澳大利亚', '大韩民国', '马来西亚']
droped = []
for area in area_name:
    for unarea in unchina:
        if unarea in area: 
            droped.append(area)
    if len(area)==2 and area not in droped: area_len_2.append(area) # 我国共有34个省级行政区域，包括23个省，5个自治区，4个直辖市，2个特别行政区。
    if len(area)==3 and area not in droped: area_len_3.append(area)
    if len(area)==4 and area not in droped: area_len_4.append(area)
    if len(area)>=5 and area not in droped: area_len_5.append(area)
print(len(droped),'\n', droped)
print(len(area_len_2),'\n', area_len_2)
print(len(area_len_3),'\n', area_len_3)
print(len(area_len_4),'\n', area_len_4)
print(len(area_len_5),'\n', area_len_5)

47 
 ['澳大利亚', '日本', '英国英格兰', '新加坡', '美国', '美国加利福尼亚州', '美国纽约州', '美国伊利诺伊州', '大韩民国', '英国', '法国', '美国马里兰州', '加拿大安大略', '加拿大不列颠哥伦比亚', '泰国', '英国苏格兰', '美国康乃狄克州', '美国德克萨斯州', '美国俄亥俄州', '美国佛罗里达州', '瑞典', '美国俄勒冈州', '马来西亚', '新西兰', '日本和歌山县', '加拿大', '加拿大艾伯塔', '日本岐阜县', '伊拉克', '美国乔治亚', '美国路易斯安那州', '爱尔兰', '美国新泽西州', '越南胡志明市', '菲律宾西米沙鄢', '菲律宾', '澳大利亚澳大利亚首都领地', '瑞士', '美国弗吉尼亚州', '安哥拉', '美国宾夕法尼亚州', '意大利皮埃蒙特', '意大利托斯卡纳', '意大利', '美国密苏里州', '美国密歇根州', '美国田纳西州']
28 
 ['北京', '上海', '重庆', '浙江', '天津', '香港', '广东', '福建', '河北', '湖南', '云南', '山东', '吉林', '河南', '贵州', '四川', '广西', '辽宁', '湖北', '山西', '江苏', '宁夏', '新疆', '甘肃', '江西', '澳门', '青海', '海南']
2 
 ['黑龙江', '内蒙古']
223 
 ['广东广州', '四川成都', '广东深圳', '湖北武汉', '江苏南京', '浙江杭州', '陕西西安', '福建福州', '江苏苏州', '安徽合肥', '山东济南', '江苏徐州', '江苏无锡', '辽宁沈阳', '山东青岛', '江西南昌', '河南郑州', '广东佛山', '湖南长沙', '云南昆明', '北京海淀', '山西太原', '广西南宁', '广东东莞', '甘肃兰州', '河南开封', '辽宁大连', '江苏南通', '浙江绍兴', '吉林长春', '辽宁盘锦', '浙江宁波', '福建厦门', '贵州贵阳', '浙江金华', '山西晋城', '浙江温州', '山东临沂', '江苏常州', '河南洛阳', '广东汕头', '江苏扬州', '山东淄博', '四川内江', '江苏泰州', '福建

上一步骤按照地理信息是否为国内以及文字长度，对数据进行了划分，有了更清晰的了解后，才能避免提取省份和城市时遇到什么抓瞎的情况。

### 省份汇总
根据百度百科词条里的内容：[省份 - 百科](https://baike.baidu.com/item/%E7%9C%81%E4%BB%BD/1635191)：`我国共有34个省级行政区域，包括23个省，5个自治区，4个直辖市，2个特别行政区。`

复制过来所有省份，先手动去掉自治区和行政区的后缀文字，再用代码去掉无关的文字与字符。

In [6]:
prolist = '北京市，天津市，上海市，重庆市，河北省，山西省，辽宁省，吉林省，江苏省，浙江省，安徽省，福建省，\
江西省，山东省，河南省，湖北省，湖南省，广东省，海南省，四川省，贵州省，云南省，陕西省，甘肃省，\
青海省，台湾省，广西，西藏，宁夏，新疆，香港，澳门，内蒙古，黑龙江省'
prolist = prolist.replace('市', '').replace('省', '').split('，')
print(len(prolist), prolist)

34 ['北京', '天津', '上海', '重庆', '河北', '山西', '辽宁', '吉林', '江苏', '浙江', '安徽', '福建', '江西', '山东', '河南', '湖北', '湖南', '广东', '海南', '四川', '贵州', '云南', '陕西', '甘肃', '青海', '台湾', '广西', '西藏', '宁夏', '新疆', '香港', '澳门', '内蒙古', '黑龙江']


## 提取城市 
根据以前的经验可知，个别点的经纬度数据可能没有内置，所以调用地图时，会无法显示并报错。所以先要确定哪些数据可用，哪些不可用，但到底 ECharts 或 pyecharts 里内置了哪些数据目前还没了解过。望知晓的朋友可以告知。

本次采取的方式是：从 ECharts 官网的[effectScatter-bmap](http://echarts.baidu.com/examples/editor.html?c=effectScatter-bmap&theme=light)示例中提取出城市列表，便于筛选。当然事先并不知道这里的数据和本次评论的数据的匹配度多高，只能“实践出真知”。

In [7]:
geoCoordMap = {'海门':[121.15,31.89],'鄂尔多斯':[109.781327,39.608266],'招远':[120.38,37.35],'舟山':[122.207216,29.985295],'齐齐哈尔':[123.97,47.33],'盐城':[120.13,33.38],'赤峰':[118.87,42.28],'青岛':[120.33,36.07],'乳山':[121.52,36.89],'金昌':[102.188043,38.520089],'泉州':[118.58,24.93],'莱西':[120.53,36.86],'日照':[119.46,35.42],'胶南':[119.97,35.88],'南通':[121.05,32.08],'拉萨':[91.11,29.97],'云浮':[112.02,22.93],'梅州':[116.1,24.55],'文登':[122.05,37.2],'上海':[121.48,31.22],'攀枝花':[101.718637,26.582347],'威海':[122.1,37.5],'承德':[117.93,40.97],'厦门':[118.1,24.46],'汕尾':[115.375279,22.786211],'潮州':[116.63,23.68],'丹东':[124.37,40.13],'太仓':[121.1,31.45],'曲靖':[103.79,25.51],'烟台':[121.39,37.52],'福州':[119.3,26.08],'瓦房店':[121.979603,39.627114],'即墨':[120.45,36.38],'抚顺':[123.97,41.97],'玉溪':[102.52,24.35],'张家口':[114.87,40.82],'阳泉':[113.57,37.85],'莱州':[119.942327,37.177017],'湖州':[120.1,30.86],'汕头':[116.69,23.39],'昆山':[120.95,31.39],'宁波':[121.56,29.86],'湛江':[110.359377,21.270708],'揭阳':[116.35,23.55],'荣成':[122.41,37.16],'连云港':[119.16,34.59],'葫芦岛':[120.836932,40.711052],'常熟':[120.74,31.64],'东莞':[113.75,23.04],'河源':[114.68,23.73],'淮安':[119.15,33.5],'泰州':[119.9,32.49],'南宁':[108.33,22.84],'营口':[122.18,40.65],'惠州':[114.4,23.09],'江阴':[120.26,31.91],'蓬莱':[120.75,37.8],'韶关':[113.62,24.84],'嘉峪关':[98.289152,39.77313],'广州':[113.23,23.16],'延安':[109.47,36.6],'太原':[112.53,37.87],'清远':[113.01,23.7],'中山':[113.38,22.52],'昆明':[102.73,25.04],'寿光':[118.73,36.86],'盘锦':[122.070714,41.119997],'长治':[113.08,36.18],'深圳':[114.07,22.62],'珠海':[113.52,22.3],'宿迁':[118.3,33.96],'咸阳':[108.72,34.36],'铜川':[109.11,35.09],'平度':[119.97,36.77],'佛山':[113.11,23.05],'海口':[110.35,20.02],'江门':[113.06,22.61],'章丘':[117.53,36.72],'肇庆':[112.44,23.05],'大连':[121.62,38.92],'临汾':[111.5,36.08],'吴江':[120.63,31.16],'石嘴山':[106.39,39.04],'沈阳':[123.38,41.8],'苏州':[120.62,31.32],'茂名':[110.88,21.68],'嘉兴':[120.76,30.77],'长春':[125.35,43.88],'胶州':[120.03336,36.264622],'银川':[106.27,38.47],'张家港':[120.555821,31.875428],'三门峡':[111.19,34.76],'锦州':[121.15,41.13],'南昌':[115.89,28.68],'柳州':[109.4,24.33],'三亚':[109.511909,18.252847],'自贡':[104.778442,29.33903],'吉林':[126.57,43.87],'阳江':[111.95,21.85],'泸州':[105.39,28.91],'西宁':[101.74,36.56],'宜宾':[104.56,29.77],'呼和浩特':[111.65,40.82],'成都':[104.06,30.67],'大同':[113.3,40.12],'镇江':[119.44,32.2],'桂林':[110.28,25.29],'张家界':[110.479191,29.117096],'宜兴':[119.82,31.36],'北海':[109.12,21.49],'西安':[108.95,34.27],'金坛':[119.56,31.74],'东营':[118.49,37.46],'牡丹江':[129.58,44.6],'遵义':[106.9,27.7],'绍兴':[120.58,30.01],'扬州':[119.42,32.39],'常州':[119.95,31.79],'潍坊':[119.1,36.62],'重庆':[106.54,29.59],'台州':[121.420757,28.656386],'南京':[118.78,32.04],'滨州':[118.03,37.36],'贵阳':[106.71,26.57],'无锡':[120.29,31.59],'本溪':[123.73,41.3],'克拉玛依':[84.77,45.59],'渭南':[109.5,34.52],'马鞍山':[118.48,31.56],'宝鸡':[107.15,34.38],'焦作':[113.21,35.24],'句容':[119.16,31.95],'北京':[116.46,39.92],'徐州':[117.2,34.26],'衡水':[115.72,37.72],'包头':[110,40.58],'绵阳':[104.73,31.48],'乌鲁木齐':[87.68,43.77],'枣庄':[117.57,34.86],'杭州':[120.19,30.26],'淄博':[118.05,36.78],'鞍山':[122.85,41.12],'溧阳':[119.48,31.43],'库尔勒':[86.06,41.68],'安阳':[114.35,36.1],'开封':[114.35,34.79],'济南':[117,36.65],'德阳':[104.37,31.13],'温州':[120.65,28.01],'九江':[115.97,29.71],'邯郸':[114.47,36.6],'临安':[119.72,30.23],'兰州':[103.73,36.03],'沧州':[116.83,38.33],'临沂':[118.35,35.05],'南充':[106.110698,30.837793],'天津':[117.2,39.13],'富阳':[119.95,30.07],'泰安':[117.13,36.18],'诸暨':[120.23,29.71],'郑州':[113.65,34.76],'哈尔滨':[126.63,45.75],'聊城':[115.97,36.45],'芜湖':[118.38,31.33],'唐山':[118.02,39.63],'平顶山':[113.29,33.75],'邢台':[114.48,37.05],'德州':[116.29,37.45],'济宁':[116.59,35.38],'荆州':[112.239741,30.335165],'宜昌':[111.3,30.7],'义乌':[120.06,29.32],'丽水':[119.92,28.45],'洛阳':[112.44,34.7],'秦皇岛':[119.57,39.95],'株洲':[113.16,27.83],'石家庄':[114.48,38.03],'莱芜':[117.67,36.19],'常德':[111.69,29.05],'保定':[115.48,38.85],'湘潭':[112.91,27.87],'金华':[119.64,29.12],'岳阳':[113.09,29.37],'长沙':[113,28.21],'衢州':[118.88,28.97],'廊坊':[116.7,39.53],'菏泽':[115.480656,35.23375],'合肥':[117.27,31.86],'武汉':[114.31,30.52],'大庆':[125.03,46.58]};
citys = list(geoCoordMap.keys())
print(citys)

['海门', '鄂尔多斯', '招远', '舟山', '齐齐哈尔', '盐城', '赤峰', '青岛', '乳山', '金昌', '泉州', '莱西', '日照', '胶南', '南通', '拉萨', '云浮', '梅州', '文登', '上海', '攀枝花', '威海', '承德', '厦门', '汕尾', '潮州', '丹东', '太仓', '曲靖', '烟台', '福州', '瓦房店', '即墨', '抚顺', '玉溪', '张家口', '阳泉', '莱州', '湖州', '汕头', '昆山', '宁波', '湛江', '揭阳', '荣成', '连云港', '葫芦岛', '常熟', '东莞', '河源', '淮安', '泰州', '南宁', '营口', '惠州', '江阴', '蓬莱', '韶关', '嘉峪关', '广州', '延安', '太原', '清远', '中山', '昆明', '寿光', '盘锦', '长治', '深圳', '珠海', '宿迁', '咸阳', '铜川', '平度', '佛山', '海口', '江门', '章丘', '肇庆', '大连', '临汾', '吴江', '石嘴山', '沈阳', '苏州', '茂名', '嘉兴', '长春', '胶州', '银川', '张家港', '三门峡', '锦州', '南昌', '柳州', '三亚', '自贡', '吉林', '阳江', '泸州', '西宁', '宜宾', '呼和浩特', '成都', '大同', '镇江', '桂林', '张家界', '宜兴', '北海', '西安', '金坛', '东营', '牡丹江', '遵义', '绍兴', '扬州', '常州', '潍坊', '重庆', '台州', '南京', '滨州', '贵阳', '无锡', '本溪', '克拉玛依', '渭南', '马鞍山', '宝鸡', '焦作', '句容', '北京', '徐州', '衡水', '包头', '绵阳', '乌鲁木齐', '枣庄', '杭州', '淄博', '鞍山', '溧阳', '库尔勒', '安阳', '开封', '济南', '德阳', '温州', '九江', '邯郸', '临安', '兰州', '沧州', '临沂', '南充', '天津', '富阳', '泰安', '诸暨', '郑州', '哈尔滨', '聊城

## 创建city 列
创建`city`列，如果`area`里包含这些城市，就返回这些城市；如果只有省份而无具体城市信息的，则返回`'pro'`（其中：`'北京', '天津', '上海', '重庆'`这些感觉也能算法城市里，纠结...），其中；如果是包含海外国家的则返回`'unchina'`；剩下的返回`'unknown'`

In [10]:
def get_city(area):
    citys = ['海门', '鄂尔多斯', '招远', '舟山', '齐齐哈尔', '盐城', '赤峰', '青岛', '乳山', '金昌', '泉州', '莱西', '日照', '胶南', '南通', '拉萨', '云浮', '梅州', '文登', '上海', '攀枝花', '威海', '承德', '厦门', '汕尾', '潮州', '丹东', '太仓', '曲靖', '烟台', '福州', '瓦房店', '即墨', '抚顺', '玉溪', '张家口', '阳泉', '莱州', '湖州', '汕头', '昆山', '宁波', '湛江', '揭阳', '荣成', '连云港', '葫芦岛', '常熟', '东莞', '河源', '淮安', '泰州', '南宁', '营口', '惠州', '江阴', '蓬莱', '韶关', '嘉峪关', '广州', '延安', '太原', '清远', '中山', '昆明', '寿光', '盘锦', '长治', '深圳', '珠海', '宿迁', '咸阳', '铜川', '平度', '佛山', '海口', '江门', '章丘', '肇庆', '大连', '临汾', '吴江', '石嘴山', '沈阳', '苏州', '茂名', '嘉兴', '长春', '胶州', '银川', '张家港', '三门峡', '锦州', '南昌', '柳州', '三亚', '自贡', '吉林', '阳江', '泸州', '西宁', '宜宾', '呼和浩特', '成都', '大同', '镇江', '桂林', '张家界', '宜兴', '北海', '西安', '金坛', '东营', '牡丹江', '遵义', '绍兴', '扬州', '常州', '潍坊', '重庆', '台州', '南京', '滨州', '贵阳', '无锡', '本溪', '克拉玛依', '渭南', '马鞍山', '宝鸡', '焦作', '句容', '北京', '徐州', '衡水', '包头', '绵阳', '乌鲁木齐', '枣庄', '杭州', '淄博', '鞍山', '溧阳', '库尔勒', '安阳', '开封', '济南', '德阳', '温州', '九江', '邯郸', '临安', '兰州', '沧州', '临沂', '南充', '天津', '富阳', '泰安', '诸暨', '郑州', '哈尔滨', '聊城', '芜湖', '唐山', '平顶山', '邢台', '德州', '济宁', '荆州', '宜昌', '义乌', '丽水', '洛阳', '秦皇岛', '株洲', '石家庄', '莱芜', '常德', '保定', '湘潭', '金华', '岳阳', '长沙', '衢州', '廊坊', '菏泽', '合肥', '武汉', '大庆']
    prolist_less = [ '河北', '山西', '辽宁', '吉林', '江苏', '浙江', '安徽', '福建', '江西', '山东', '河南', '湖北', '湖南', '广东', '海南', '四川', '贵州', '云南', '陕西', '甘肃', '青海', '广西', '西藏', '宁夏', '新疆', '香港', '澳门', '台湾', '内蒙古', '黑龙江']
    # 此处 prolist_less 个人喜欢，所以不包含：'北京', '天津', '上海', '重庆' ...
    unchinas = ['英国','美国','日本','瑞士','法国','瑞典','越南','泰国','意大利','加拿大','菲律宾','新加坡','新西兰','伊拉克','爱尔兰','安哥拉','澳大利亚', '大韩民国', '马来西亚']
    for city in citys:
        if city in area:
            return city
    for pro in prolist_less:
        if pro == area:
            return "pro"
    for unchina in unchinas:
        if unchina in area:
            return "unchina"
    return "unknown"
df['city'] = df.area.apply(get_city)
df[['area','city']]

,area,city
0,青海海南藏族自治州,unknown
1,安徽合肥,合肥
2,山西太原,太原
3,广东广州,广州
4,广东深圳,深圳
5,湖南,pro
6,四川成都,成都
7,浙江,pro
8,广西南宁,南宁
9,浙江杭州,杭州


`unknown` 的地理信息有以下这些，即非海外，也不是省份，而且不包含 ECharts 那些城市。但几乎都是省份+城市的格式，所以提取城市比较方便。

In [11]:
df_unknown = df[['area','city']][df['city']=='unknown']
print(df_unknown.shape)
unknown_city = list(set(df_unknown.area.values.tolist()))
print(len(unknown_city),unknown_city)

(247, 2)
104 ['四川遂宁', '新疆塔城地区', '山西吕梁', '湖北荆门', '湖北随州', '河南漯河', '宁夏固原', '湖南湘西土家族苗族自治州', '甘肃张掖', '安徽蚌埠', '辽宁朝阳', '新疆石河子', '安徽滁州', '安徽黄山', '甘肃定西', '河南南阳', '四川雅安', '广西钦州', '云南怒江傈僳族自治州', '甘肃天水', '新疆巴音郭楞蒙古自治州', '湖北恩施土家族苗族自治州', '福建宁德', '辽宁辽阳', '江西萍乡', '福建漳州', '福建三明', '山西运城', '辽宁阜新', '青海海南藏族自治州', '安徽淮北', '内蒙古呼伦贝尔', '辽宁铁岭', '河南周口', '湖南娄底', '内蒙古通辽', '湖南邵阳', '福建南平', '云南昭通', '新疆伊犁哈萨克自治州', '江西景德镇', '河南鹤壁', '安徽池州', '宁夏吴忠', '福建龙岩', '湖北咸宁', '新疆昌吉回族自治州', '四川广安', '湖北襄阳', '四川资阳', '安徽阜阳', '陕西汉中', '云南大理白族自治州', '广西玉林', '四川眉山', '河南信阳', '内蒙古乌兰察布', '河南新乡', '山西晋中', '河南商丘', '贵州毕节', '四川内江', '四川达州', '湖北黄石', '黑龙江绥化', '安徽安庆', '湖北黄冈', '四川乐山', '湖南永州', '内蒙古兴安盟', '湖南益阳', '江西鹰潭', '湖南衡阳', '云南红河哈尼族彝族自治州', '湖南怀化', '江西宜春', '云南临沧', '江西上饶', '甘肃武威', '广西梧州', '陕西商洛', '江西抚州', '安徽六安', '广西崇左', '山西晋城', '湖北孝感', '黑龙江鸡西', '广西河池', '黑龙江伊春', '江西赣州', '云南保山', '河南驻马店', '河南濮阳', '山西朔州', '福建莆田', '甘肃白银', '安徽淮南', '安徽铜陵', '河南许昌', '四川广元', '湖南郴州', '云南普洱', '青海海东', '广西百色']


遍历`unknown_city`，并去掉省份的前缀，然后加到`unkwns`列表中

In [12]:
prolist_less = [ '河北', '山西', '辽宁', '吉林', '江苏', '浙江', '安徽', '福建', '江西', '山东', '河南', '湖北', '湖南', '广东', '海南', '四川', '贵州', '云南', '陕西', '甘肃', '青海', '广西', '西藏', '宁夏', '新疆', '香港', '澳门', '内蒙古', '黑龙江']
unkwns = []
others = []
for unkwn in unknown_city:
    for pro in prolist_less:
        if pro in unkwn: #  '青海海南藏族自治州'
            unkwn = unkwn.replace(pro, '')
            unkwns.append(unkwn)
    if unkwn not in unkwns:
        others.append(unkwn)
print(len(unkwns))
print(unkwns) # '青海藏族自治州', '藏族自治州'
print(len(others))
print(others)

105
['遂宁', '塔城地区', '吕梁', '荆门', '随州', '漯河', '固原', '湘西土家族苗族自治州', '张掖', '蚌埠', '朝阳', '石河子', '滁州', '黄山', '定西', '南阳', '雅安', '钦州', '怒江傈僳族自治州', '天水', '巴音郭楞蒙古自治州', '恩施土家族苗族自治州', '宁德', '辽阳', '萍乡', '漳州', '三明', '运城', '阜新', '青海藏族自治州', '藏族自治州', '淮北', '呼伦贝尔', '铁岭', '周口', '娄底', '通辽', '邵阳', '南平', '昭通', '伊犁哈萨克自治州', '景德镇', '鹤壁', '池州', '吴忠', '龙岩', '咸宁', '昌吉回族自治州', '广安', '襄阳', '资阳', '阜阳', '汉中', '大理白族自治州', '玉林', '眉山', '信阳', '乌兰察布', '新乡', '晋中', '商丘', '毕节', '内江', '达州', '黄石', '绥化', '安庆', '黄冈', '乐山', '永州', '兴安盟', '益阳', '鹰潭', '衡阳', '红河哈尼族彝族自治州', '怀化', '宜春', '临沧', '上饶', '武威', '梧州', '商洛', '抚州', '六安', '崇左', '晋城', '孝感', '鸡西', '河池', '伊春', '赣州', '保山', '驻马店', '濮阳', '朔州', '莆田', '白银', '淮南', '铜陵', '许昌', '广元', '郴州', '普洱', '海东', '百色']
0
[]


## 汇总所有城市
将`unkwns`里面含有xxx自治州的都手动去掉后缀，然后将`unkwns`和`citys`合并到一起，`all_citys`就是我们要统计和进行地图可视化的所有城市数据了。以上就是完成了城市数据的提取，读者可自行按照自己的思路去实现，不必局限于我的方法。

In [13]:
citys = ['海门', '鄂尔多斯', '招远', '舟山', '齐齐哈尔', '盐城', '赤峰', '青岛', '乳山', '金昌', '泉州', '莱西', '日照', '胶南', '南通', '拉萨', '云浮', '梅州', '文登', '上海', '攀枝花', '威海', '承德', '厦门', '汕尾', '潮州', '丹东', '太仓', '曲靖', '烟台', '福州', '瓦房店', '即墨', '抚顺', '玉溪', '张家口', '阳泉', '莱州', '湖州', '汕头', '昆山', '宁波', '湛江', '揭阳', '荣成', '连云港', '葫芦岛', '常熟', '东莞', '河源', '淮安', '泰州', '南宁', '营口', '惠州', '江阴', '蓬莱', '韶关', '嘉峪关', '广州', '延安', '太原', '清远', '中山', '昆明', '寿光', '盘锦', '长治', '深圳', '珠海', '宿迁', '咸阳', '铜川', '平度', '佛山', '海口', '江门', '章丘', '肇庆', '大连', '临汾', '吴江', '石嘴山', '沈阳', '苏州', '茂名', '嘉兴', '长春', '胶州', '银川', '张家港', '三门峡', '锦州', '南昌', '柳州', '三亚', '自贡', '吉林', '阳江', '泸州', '西宁', '宜宾', '呼和浩特', '成都', '大同', '镇江', '桂林', '张家界', '宜兴', '北海', '西安', '金坛', '东营', '牡丹江', '遵义', '绍兴', '扬州', '常州', '潍坊', '重庆', '台州', '南京', '滨州', '贵阳', '无锡', '本溪', '克拉玛依', '渭南', '马鞍山', '宝鸡', '焦作', '句容', '北京', '徐州', '衡水', '包头', '绵阳', '乌鲁木齐', '枣庄', '杭州', '淄博', '鞍山', '溧阳', '库尔勒', '安阳', '开封', '济南', '德阳', '温州', '九江', '邯郸', '临安', '兰州', '沧州', '临沂', '南充', '天津', '富阳', '泰安', '诸暨', '郑州', '哈尔滨', '聊城', '芜湖', '唐山', '平顶山', '邢台', '德州', '济宁', '荆州', '宜昌', '义乌', '丽水', '洛阳', '秦皇岛', '株洲', '石家庄', '莱芜', '常德', '保定', '湘潭', '金华', '岳阳', '长沙', '衢州', '廊坊', '菏泽', '合肥', '武汉', '大庆']
unkwns = ['眉山', '漯河', '滁州', '景德镇', '随州', '遂宁', '三明', '乌兰察布', '吕梁', '莆田', '河池', '定西', '信阳', '阜新', '广元', '汉中', '辽阳', '邵阳', '萍乡', '晋中', '商洛', '乐山', '石河子', '白银', '池州', '内江', '恩施', '益阳', '伊犁', '张掖', '运城', '绥化', '崇左', '南阳', '达州', '海东', '伊春', '黄石', '龙岩', '蚌埠', '上饶', '驻马店', '保山', '呼伦贝尔', '安庆', '淮北', '周口', '孝感', '临沧', '固原', '钦州', '漳州', '巴音郭楞', '宜春', '朝阳', '鹰潭', '娄底', '雅安', '青海', '怀化', '郴州', '许昌', '红河', '赣州', '朔州', '普洱', '吴忠', '新乡', '资阳', '鸡西', '昌吉', '荆门', '六安', '黄山', '商丘', '武威', '抚州', '湘西', '怒江', '铁岭', '百色', '淮南', '广安', '铜陵', '濮阳', '天水', '襄阳', '南平', '阜阳', '鹤壁', '塔城', '宁德', '大理', '梧州', '衡阳', '晋城', '昭通', '黄冈', '永州', '毕节', '通辽', '玉林', '兴安盟', '咸宁']
all_citys = citys + unkwns
print(len(all_citys), all_citys)

294 ['海门', '鄂尔多斯', '招远', '舟山', '齐齐哈尔', '盐城', '赤峰', '青岛', '乳山', '金昌', '泉州', '莱西', '日照', '胶南', '南通', '拉萨', '云浮', '梅州', '文登', '上海', '攀枝花', '威海', '承德', '厦门', '汕尾', '潮州', '丹东', '太仓', '曲靖', '烟台', '福州', '瓦房店', '即墨', '抚顺', '玉溪', '张家口', '阳泉', '莱州', '湖州', '汕头', '昆山', '宁波', '湛江', '揭阳', '荣成', '连云港', '葫芦岛', '常熟', '东莞', '河源', '淮安', '泰州', '南宁', '营口', '惠州', '江阴', '蓬莱', '韶关', '嘉峪关', '广州', '延安', '太原', '清远', '中山', '昆明', '寿光', '盘锦', '长治', '深圳', '珠海', '宿迁', '咸阳', '铜川', '平度', '佛山', '海口', '江门', '章丘', '肇庆', '大连', '临汾', '吴江', '石嘴山', '沈阳', '苏州', '茂名', '嘉兴', '长春', '胶州', '银川', '张家港', '三门峡', '锦州', '南昌', '柳州', '三亚', '自贡', '吉林', '阳江', '泸州', '西宁', '宜宾', '呼和浩特', '成都', '大同', '镇江', '桂林', '张家界', '宜兴', '北海', '西安', '金坛', '东营', '牡丹江', '遵义', '绍兴', '扬州', '常州', '潍坊', '重庆', '台州', '南京', '滨州', '贵阳', '无锡', '本溪', '克拉玛依', '渭南', '马鞍山', '宝鸡', '焦作', '句容', '北京', '徐州', '衡水', '包头', '绵阳', '乌鲁木齐', '枣庄', '杭州', '淄博', '鞍山', '溧阳', '库尔勒', '安阳', '开封', '济南', '德阳', '温州', '九江', '邯郸', '临安', '兰州', '沧州', '临沂', '南充', '天津', '富阳', '泰安', '诸暨', '郑州', '哈尔滨',

## 更新 city 列
更新下数据里的`city`列，含有这些城市的就返回对应城市，否则不论国外还是省份都统一用`others`表示

In [20]:
def get_city(area):
    all_citys = ['海门', '鄂尔多斯', '招远', '舟山', '齐齐哈尔', '盐城', '赤峰', '青岛', '乳山', '金昌', '泉州', '莱西', 
                 '日照', '胶南', '南通', '拉萨', '云浮', '梅州', '文登', '上海', '攀枝花', '威海', '承德', '厦门', '汕尾', 
                 '潮州', '丹东', '太仓', '曲靖', '烟台', '福州', '瓦房店', '即墨', '抚顺', '玉溪', '张家口', '阳泉', '莱州',
                 '湖州', '汕头', '昆山', '宁波', '湛江', '揭阳', '荣成', '连云港', '葫芦岛', '常熟', '东莞', '河源', '淮安', 
                 '泰州', '南宁', '营口', '惠州', '江阴', '蓬莱', '韶关', '嘉峪关', '广州', '延安', '太原', '清远', '中山', 
                 '昆明', '寿光', '盘锦', '长治', '深圳', '珠海', '宿迁', '咸阳', '铜川', '平度', '佛山', '海口', '江门', 
                 '章丘', '肇庆', '大连', '临汾', '吴江', '石嘴山', '沈阳', '苏州', '茂名', '嘉兴', '长春', '胶州', '银川', 
                 '张家港', '三门峡', '锦州', '南昌', '柳州', '三亚', '自贡', '吉林', '阳江', '泸州', '西宁', '宜宾',
                 '呼和浩特', '成都', '大同', '镇江', '桂林', '张家界', '宜兴', '北海', '西安', '金坛', '东营', '牡丹江', 
                 '遵义', '绍兴', '扬州', '常州', '潍坊', '重庆', '台州', '南京', '滨州', '贵阳', '无锡', '本溪', '克拉玛依', 
                 '渭南', '马鞍山', '宝鸡', '焦作', '句容', '北京', '徐州', '衡水', '包头', '绵阳', '乌鲁木齐', '枣庄', 
                 '杭州', '淄博', '鞍山', '溧阳', '库尔勒', '安阳', '开封', '济南', '德阳', '温州', '九江', '邯郸', '临安', 
                 '兰州', '沧州', '临沂', '南充', '天津', '富阳', '泰安', '诸暨', '郑州', '哈尔滨', '聊城', '芜湖', '唐山', 
                 '平顶山', '邢台', '德州', '济宁', '荆州', '宜昌', '义乌', '丽水', '洛阳', '秦皇岛', '株洲', '石家庄', 
                 '莱芜', '常德', '保定', '湘潭', '金华', '岳阳', '长沙', '衢州', '廊坊', '菏泽', '合肥', '武汉', '大庆', 
                 '眉山', '漯河', '滁州', '景德镇', '随州', '遂宁', '三明', '乌兰察布', '吕梁', '莆田', '河池', '定西', 
                 '信阳', '阜新', '广元', '汉中', '辽阳', '邵阳', '萍乡', '晋中', '商洛', '乐山', '石河子', '白银', '池州', 
                 '内江', '恩施', '益阳', '伊犁', '张掖', '运城', '绥化', '崇左', '南阳', '达州', '海东', '伊春', '黄石', 
                 '龙岩', '蚌埠', '上饶', '驻马店', '保山', '呼伦贝尔', '安庆', '淮北', '周口', '孝感', '临沧', '固原', '钦州', 
                 '漳州', '巴音郭楞', '宜春', '朝阳', '鹰潭', '娄底', '雅安', '青海', '怀化', '郴州', '许昌', '红河', '赣州', 
                 '朔州', '普洱', '吴忠', '新乡', '资阳', '鸡西', '昌吉', '荆门', '六安', '黄山', '商丘', '武威', '抚州', 
                 '湘西', '怒江', '铁岭', '百色', '淮南', '广安', '铜陵', '濮阳', '天水', '襄阳', '南平', '阜阳', '鹤壁', 
                 '塔城', '宁德', '大理', '梧州', '衡阳', '晋城', '昭通', '黄冈', '永州', '毕节', '通辽', '玉林', '兴安盟', 
                 '咸宁']
    for city in all_citys:
        if city in area:
            return city
    return "other"
df['city'] = df.area.apply(get_city)
df[['area','city']]

,area,city
0,青海海南藏族自治州,青海
1,安徽合肥,合肥
2,山西太原,太原
3,广东广州,广州
4,广东深圳,深圳
5,湖南,other
6,四川成都,成都
7,浙江,other
8,广西南宁,南宁
9,浙江杭州,杭州


## 城市分布

In [21]:
city_count = df.groupby('city')['city'].count().sort_values(ascending=False)
city_count

city
other    462
北京       352
上海       284
广州       176
成都       136
深圳       131
武汉       113
重庆        96
南京        96
杭州        87
福州        73
西安        73
苏州        64
合肥        52
天津        49
济南        44
无锡        42
徐州        42
沈阳        40
青岛        40
郑州        39
南昌        39
佛山        38
长沙        36
昆明        31
太原        30
南宁        29
兰州        29
东莞        28
呼和浩特      27
        ... 
景德镇        1
焦作         1
朔州         1
潮州         1
梅州         1
永州         1
汕尾         1
河源         1
渭南         1
红河         1
怒江         1
怀化         1
孝感         1
商洛         1
固原         1
塔城         1
大理         1
天水         1
蚌埠         1
娄底         1
葫芦岛        1
绥化         1
定西         1
莱芜         1
荆门         1
广元         1
广安         1
肇庆         1
张掖         1
菏泽         1
Name: city, Length: 257, dtype: int64

In [22]:
from pyecharts import Bar
#bar.use_theme("macarons")
bar = Bar("城市分布")
bar.add("城市", city_count.index[:30], city_count.values[:30],is_label_show=True,xaxis_interval=0,xaxis_rotate=-45)
bar

组合成元组，方便后面地图可视化时传入数据

In [24]:
city_data = list(zip(city_count.index, city_count.values))
city_data

[('other', 462),
 ('北京', 352),
 ('上海', 284),
 ('广州', 176),
 ('成都', 136),
 ('深圳', 131),
 ('武汉', 113),
 ('重庆', 96),
 ('南京', 96),
 ('杭州', 87),
 ('福州', 73),
 ('西安', 73),
 ('苏州', 64),
 ('合肥', 52),
 ('天津', 49),
 ('济南', 44),
 ('无锡', 42),
 ('徐州', 42),
 ('沈阳', 40),
 ('青岛', 40),
 ('郑州', 39),
 ('南昌', 39),
 ('佛山', 38),
 ('长沙', 36),
 ('昆明', 31),
 ('太原', 30),
 ('南宁', 29),
 ('兰州', 29),
 ('东莞', 28),
 ('呼和浩特', 27),
 ('开封', 24),
 ('南通', 22),
 ('大连', 22),
 ('哈尔滨', 22),
 ('石家庄', 22),
 ('绍兴', 20),
 ('吉林', 17),
 ('宁波', 14),
 ('厦门', 14),
 ('盘锦', 14),
 ('长春', 14),
 ('贵阳', 13),
 ('金华', 13),
 ('临沂', 12),
 ('晋城', 12),
 ('温州', 12),
 ('扬州', 11),
 ('内江', 11),
 ('汕头', 11),
 ('常州', 11),
 ('洛阳', 11),
 ('乌鲁木齐', 11),
 ('淄博', 11),
 ('泉州', 10),
 ('中山', 10),
 ('泰州', 10),
 ('烟台', 9),
 ('嘉兴', 9),
 ('包头', 9),
 ('湖州', 9),
 ('济宁', 9),
 ('宜昌', 9),
 ('廊坊', 8),
 ('海口', 8),
 ('惠州', 8),
 ('潍坊', 8),
 ('连云港', 8),
 ('保定', 8),
 ('绵阳', 7),
 ('淮北', 7),
 ('柳州', 7),
 ('九江', 7),
 ('大同', 7),
 ('襄阳', 7),
 ('台州', 7),
 ('邯郸', 7),
 ('周口', 7),
 ('

## 地图异常
一些城市的经纬度数据没有内置到地图里，所以会报错，需要去掉这部分城市数据再进行可视化

In [25]:
from pyecharts import Geo

geo = Geo("城市分布情况", "data from SinaNews", title_color="#fff",
          title_pos="center", width=800,
          height=600, background_color='#404a59')
attr, value = geo.cast(city_data)
geo.add("城市分布情况", attr, value, visual_range=[0, 360], visual_text_color="#fff",
        symbol_size=10, is_visualmap=True)
geo

ValueError: No coordinate is specified for other

报错信息说明 pyecharts 里没有`other`的经纬度信息。

创建`nocoor_list`列表，用于存储所有报错的城市，反复运行下面两个 cell 的代码，根据地图报错提醒，不断添加城市到`nocoor_list`列表中，就能得到全部的无经纬度的城市。并将剩下没报错的都存储到`city_data` 列表中。

In [30]:
city_data = []
nocoor_list = [
    'other','青海','达州','崇左','巴音郭楞','普洱','池州','湘西','伊犁','乌兰察布','临沧','海东','眉山',
    '怒江','固原','广安'
             ]
for item in zip(city_count.index, city_count.values):
    if item[0] in nocoor_list:
        continue  # 跳出循环：即遇到不存在经纬度的城市就跳过，不用存储到 city_data 里
    city_data.append(item)
city_data

[('北京', 352),
 ('上海', 284),
 ('广州', 176),
 ('成都', 136),
 ('深圳', 131),
 ('武汉', 113),
 ('重庆', 96),
 ('南京', 96),
 ('杭州', 87),
 ('福州', 73),
 ('西安', 73),
 ('苏州', 64),
 ('合肥', 52),
 ('天津', 49),
 ('济南', 44),
 ('无锡', 42),
 ('徐州', 42),
 ('沈阳', 40),
 ('青岛', 40),
 ('郑州', 39),
 ('南昌', 39),
 ('佛山', 38),
 ('长沙', 36),
 ('昆明', 31),
 ('太原', 30),
 ('南宁', 29),
 ('兰州', 29),
 ('东莞', 28),
 ('呼和浩特', 27),
 ('开封', 24),
 ('南通', 22),
 ('大连', 22),
 ('哈尔滨', 22),
 ('石家庄', 22),
 ('绍兴', 20),
 ('吉林', 17),
 ('宁波', 14),
 ('厦门', 14),
 ('盘锦', 14),
 ('长春', 14),
 ('贵阳', 13),
 ('金华', 13),
 ('临沂', 12),
 ('晋城', 12),
 ('温州', 12),
 ('扬州', 11),
 ('内江', 11),
 ('汕头', 11),
 ('常州', 11),
 ('洛阳', 11),
 ('乌鲁木齐', 11),
 ('淄博', 11),
 ('泉州', 10),
 ('中山', 10),
 ('泰州', 10),
 ('烟台', 9),
 ('嘉兴', 9),
 ('包头', 9),
 ('湖州', 9),
 ('济宁', 9),
 ('宜昌', 9),
 ('廊坊', 8),
 ('海口', 8),
 ('惠州', 8),
 ('潍坊', 8),
 ('连云港', 8),
 ('保定', 8),
 ('绵阳', 7),
 ('淮北', 7),
 ('柳州', 7),
 ('九江', 7),
 ('大同', 7),
 ('襄阳', 7),
 ('台州', 7),
 ('邯郸', 7),
 ('周口', 7),
 ('芜湖', 6),
 ('唐山', 

## pyecharts 城市分布地图
顺利绘制出城市分布地图。

大家一定要自己梳理下整个城市提取和可视化的过程。最开始也是古柳根据以往实践的经验，再不知道 pyecharts 都支持哪些城市（可自行搜索看看有没有官方说明）的情况下，选择去官网复制了那一堆城市数据，然后一步步完成了下来，相对比较顺利。

In [31]:
from pyecharts import Geo

geo = Geo("城市分布情况", "data from SinaNews", title_color="#fff",
          title_pos="center", width=800,
          height=600, background_color='#404a59')
attr, value = geo.cast(city_data)
geo.add("", attr, value, visual_range=[0, 360], visual_text_color="#fff",
        symbol_size=10, is_visualmap=True)
geo

## 小结
篇幅所限，后面动态地图可视化内容放到下一篇文章里。先给点提醒用的是[WEB 服务API - 百度地图](http://lbsyun.baidu.com/index.php?title=webapi/ip-api)获取`area`列地理位置的经纬度，然后导出包含经纬度两列的完整数据到 csv，再根据[（送福利）BDP绘制微博转发动态热力图](https://zhuanlan.zhihu.com/p/29557747)一文，轻松实现预期效果。
<img src='images/heat-map-BDP-1h-11FPS.gif'> 